In [1]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm
import numpy as np

zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-base-zeroshot-v1.1-all-33")

In [71]:
df = pd.read_csv('../../cylindrical shape or conical shape or cushioning.csv')
df.sort_values(by='score', ascending=False).head(50)

,dimension,noun,score
1609,cylindrical shape or conical shape or cushioning,cylinder,0.992385
468,cylindrical shape or conical shape or cushioning,barrel,0.976635
6150,cylindrical shape or conical shape or cushioning,topper,0.976309
6276,cylindrical shape or conical shape or cushioning,tube,0.976017
2786,cylindrical shape or conical shape or cushioning,hammock,0.975951
1315,cylindrical shape or conical shape or cushioning,cone,0.975134
287,cylindrical shape or conical shape or cushioning,armchair,0.974613
780,cylindrical shape or conical shape or cushioning,bumper,0.974068
1800,cylindrical shape or conical shape or cushioning,disc,0.970789
2831,cylindrical shape or conical shape or cushioning,hazmat,0.970576


In [45]:
with open('../../things_66.txt', 'r') as file:
    lines = file.readlines()
    dimensions = [line.strip() for line in lines]

with open('../../data/laion_nouns.txt', 'r') as file:
    lines = file.readlines()
    nouns = [line.strip() for line in lines]

In [51]:
hypothesis_template = "This word can be categorized as {}"
dimension = 'white color'


f = 'laion_nouns_dimensions.csv'

data = []

for dimension in dimensions:
    candidate_labels=[f'{t} {dimension}' for t in ["related to", "not related to"]]
    for noun in tqdm(nouns):
        result = zeroshot_classifier(noun, candidate_labels, hypothesis_template=hypothesis_template)
        idx = np.where(np.array(result['labels']) == candidate_labels[0])[0][0]
        score = result['scores'][idx]
        data.append([dimension, noun, score])
    break
#     data_n = [noun]
#     for label in candidate_labels:
#         idx = np.where(np.array(result['labels']) == label)[0][0]
#         data_n.append(f"{result['scores'][idx]:.3f}")
#     data.append(data_n)
#     break

# data = pd.DataFrame(data, columns=['noun'] + candidate_labels)
# data.to_csv(f, index=False)

 14%|█▍        | 942/6725 [06:10<37:56,  2.54it/s]  


KeyboardInterrupt: 

In [40]:
text = "milk"
hypothesis_template = "This word is {}"
dimension = 'white color'
candidate_labels=[f'{t} {dimension}' for t in ["related to", "not related to"]]

result = zeroshot_classifier(text, candidate_labels, hypothesis_template=hypothesis_template)
label = result['labels'][0]
score = result['scores'][0]
result

{'sequence': 'milk',
 'labels': ['related to white color', 'not related to white color'],
 'scores': [0.6522828340530396, 0.34771713614463806]}

In [218]:
hypothesis_template = "This word can be categorized as {}"
candidate_labels=["human", "animal", "plant", "vehicle", "food", "clothing", "body part", "furniture", "electronics", "item, tool or appliance", "manmade area", "natural area", "event", "feeling or emotion"]


with open('laion_nouns.txt', 'r') as file:
    lines = file.readlines()
    nouns = [line.strip() for line in lines]

f = 'laion_nouns_classification.csv'

data = []

for noun in tqdm(nouns):
    result = zeroshot_classifier(noun, candidate_labels, hypothesis_template=hypothesis_template)
    data_n = [noun]
    for label in candidate_labels:
        idx = np.where(np.array(result['labels']) == label)[0][0]
        data_n.append(f"{result['scores'][idx]:.3f}")
    data.append(data_n)
    break

data = pd.DataFrame(data, columns=['noun'] + candidate_labels)
data.to_csv(f, index=False)

  0%|          | 0/6726 [00:02<?, ?it/s]
